#  Train a GPT-2 Model with AITEXTGEN

This tutorial explores fine-tuning and text generation with the GPT-2 language model. More information on GPT-2 can be found here: [Better Language Models and Their Implications](https://openai.com/blog/better-language-models/)

<br>

`AITEXTGEN` is authored by [Max Woolf](https://minimaxir.com). For more info you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/). This is an evolution of Woolf's earlier project [GPT2-SIMPLE](https://github.com/minimaxir/gpt-2-simple). `AITEXTGEN` also takes advantage of processes authored by [HuggingFace](https://huggingface.co/), so it may be interesting to research their [NLP Models](https://huggingface.co/models) and the [Transformers package](https://huggingface.co/docs/transformers/installation).

<br>

Colab Setup:

Make sure GPU is enabled, go to Edit->Notebook settings->Hardware Accelerator->GPU

Note: Free Colab runtimes can vary greatly in duration, from 4-12 hours before diconnecting, so we'll discuss later how to save your progress at regular intervals. Colab Plus is a paid service that provides longer runtimes with more RAM and better GPUs.



#Environment setup
This cell installs/imports `AITEXTGEN` and some utilities for interfacing with your Google Drive. Much of the following tutorial is copied from Woolf's Colab demo: [Finetune OpenAI's 124M GPT-2 model (or GPT Neo) on your own dataset (GPU)](https://colab.research.google.com/drive/15qBZx5y9rdaQSyWpsreMDnTiZ5IlN0zD?usp=sharing).

In [ ]:
!pip install aitextgen
!pip install pytorch-lightning==1.7.7

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

## Loading GPT-2 or another base model

If you're finetuning a model with your own text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M`: the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, 3GB on disk.

You can also finetune alternative base models such as [GPT Neo](https://www.eleuther.ai/projects/gpt-neo/), which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.

In this case, *many* of the models hosted on [HuggingFace](https://huggingface.co/models) may be used as a base model for finetuning. With this in mind, even non-English models may be finetuned with this code. 

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

# Comment out the above line and uncomment the below line to use alternate base models such as GPT Neo.
# ai = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*. Running this cell will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code)

<br>

Alternately, this code can be run from any Colab notebook to mount your drive
<br>

`from google.colab import drive`
<br>
`drive.mount('/content/drive')`

In [ ]:
mount_gdrive()

## IMPORT DATASET
## Option 1
## Downloading a Text File Dataset for Training
In this case the file is A Tale of Two Cities (Charles Dickens) from [Project Gutenberg](https://www.gutenberg.org/). To change the dataset that GPT-2 will fine-tune on, change this URL to another .txt file, and change the corresponding `file_name` variable later in the notebook.

Keep in mind that this *must* be an 8-bit, plain text .txt file

98/98-0.txt - Dickens - A Tale of Two Cities - 139k words
<br>
84/84-0.txt - Shelley - Frankenstein - 78k words
<br>
996/996-0.txt - Cervantes - Don Quixote - 430k words

In [ ]:
!wget https://www.gutenberg.org/files/98/98-0.txt

In [ ]:
file_name = "98-0.txt"

## Option 2
## Uploading a Text File to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can *Upload to session storage*:

![alt text](https://i.imgur.com/w3wvHhR.png)

**Make sure to re-define the `file_name` variable to your dataset's name

In [ ]:
file_name = "your_dataset.txt"

## Option 3
## Transferring a Text File from Google Drive

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM. ONLY run this line, if your named text file is in the root of your personal Google Drive.

**Make sure to first re-define the `file_name` variable to your dataset's name

In [ ]:
file_name = "your_dataset.txt"

In [ ]:
copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [ ]:
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=500,
         generate_every=500,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

You're done! 
<br>
At this time it may be useful ro rename the `trained_model` folder to something unique, pehaps according to the `file_name` and `num_steps` of your finetuned model.
<br> 
Likewise, if you set `save_gdrive=True` you will find a new folder in your Google Drive formatted like `ATG_YYYYMMDD_HHMMSS`. Perhaps rename this to something logical.

<br>
Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

Make sure to change `my_model_folder` to your custom model's folder.

In [ ]:
!cp -r /content/drive/MyDrive/my_model_folder /content/
#change my_model_folder to your custom model's name

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder. Otherwise, change `trained_model` to your renamed folder.

In [ ]:
ai = aitextgen(model_folder="trained_model", to_gpu=True)

`generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate()

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [ ]:
ai.generate(n=5,
            prompt="In the forest",
            max_length=256,
            temperature=1.0,
            top_p=0.9)

# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.